In [17]:
import h5py
import os
import tables
import numpy as np
import multiprocessing
import time

In [18]:
inputfolder="/home/pablo/github/DeepIceLearning/notebooks/h5"
outputfolder="/home/pablo/github/DeepIceLearning/notebooks/h5_merged"
n=2  #number of output files (also simultaneous cpu usage)

In [19]:
print("The computer has:", multiprocessing.cpu_count(),"CPU.","You are using:",n)

The computer has: 4 CPU. You are using: 2


In [20]:
File_list=os.listdir(inputfolder)   #np.sort
Full_path_file_list = [os.path.join(inputfolder, path) for path in  File_list ]
Splited_Full_path_file_list=np.array_split(Full_path_file_list,n)
outputfile_list=[outputfolder+"/h5_merged{}.h5".format(i) for i in range(n) ]

In [5]:
def merge(file_and_output_file):
    file_list,output_file=file_and_output_file
    input_shape = [10, 10, 60]
    FILTERS = tables.Filters(complib='zlib', complevel=9)

    hf1 = h5py.File(file_list[0], 'r')
    dtype=hf1["reco_vals"].dtype
    keys = list(hf1.keys())

    with tables.open_file(output_file, mode="w", title="Events for training the NN", filters=FILTERS) as h5file:

        input_features = []
        for okey in keys[:-1]:
            feature = h5file.create_earray(
                    h5file.root, okey, tables.Float64Atom(),
                    (0, input_shape[0], input_shape[1], input_shape[2], 1),
                    title=okey)
            feature.flush()
            input_features.append(feature)
        reco_vals = tables.Table(h5file.root, 'reco_vals', description=dtype)
        h5file.root._v_attrs.shape = input_shape
        hf1.close()
        it=0
        s=np.size(file_list)
        for file in file_list:
            print('Open {}'.format(file),it/s*100,"%")
            it+=1
            one_h5file = h5py.File(file, 'r')
            num_events = len(one_h5file["reco_vals"])
            for k in range(num_events):
                for i, okey in enumerate(keys[:-1]):
                    input_features[i].append(np.expand_dims(one_h5file[okey][k], axis=0))
                reco_vals.append(np.atleast_1d(one_h5file["reco_vals"][k]))
            for inp_feature in input_features:
                    inp_feature.flush()
            reco_vals.flush()
        one_h5file.close()

    print("Finished.")

In [10]:
file_and_output_file=[(Splited_Full_path_file_list[i],outputfile_list[i]) for i in range(n)]

ti = time.time()
p = multiprocessing.Pool(n)
p.map(merge,file_and_output_file)

p.close()
p.join()
tf=time.time()-ti
print("Total time elapsed:", tf,"s")

Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_29.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_16.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_26.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_20.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_27.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_3.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_6.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_28.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_17.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_23.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_25.h5 33.33333333333333 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_8.h5 33.33333333333333 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_4.h5 33.33333333333333 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5

In [6]:
ti = time.time()
merge((Splited_Full_path_file_list[0],outputfile_list[0]))
tf=time.time()-ti
print("Total time elapsed:", tf,"s")

Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_29.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_24.h5 6.666666666666667 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_5.h5 13.333333333333334 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_6.h5 20.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_21.h5 26.666666666666668 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_12.h5 33.33333333333333 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_3.h5 40.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_4.h5 46.666666666666664 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_7.h5 53.333333333333336 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_16.h5 60.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_0.h5 66.66666666666666 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_18.h5 73.33333333333333 %
Open /home/pablo/github/DeepIceLearning/noteboo

In [23]:
#divide and conquer is not a good strategy here
ti = time.time()
merge((np.array_split(Splited_Full_path_file_list[0],n)[0],outputfile_list[0]))
tf=time.time()-ti
print("Total time elapsed:", tf,"s")
merge((np.array_split(Splited_Full_path_file_list[0],n)[1],outputfile_list[1]))
tf=time.time()-ti
print("Total time elapsed:", tf,"s")
merge((np.array([outputfile_list[0],outputfile_list[1]]),'/home/pablo/github/DeepIceLearning/notebooks/h5_merged/h5_merged3.h5'))
tf=time.time()-ti
print("Total time elapsed:", tf,"s")

Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_29.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_24.h5 12.5 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_5.h5 25.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_6.h5 37.5 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_21.h5 50.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_12.h5 62.5 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_3.h5 75.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_4.h5 87.5 %
Finished.
Total time elapsed: 57.91114807128906 s
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_7.h5 0.0 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_16.h5 14.285714285714285 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_0.h5 28.57142857142857 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_18.h5 42.857142857142854 %
Open /home/pablo/github/DeepIceLearning/notebooks/h5/File_23.h5 5